# TODO

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1AoolDYePUpPkRCKIu0cP9zV7lX5QGD3Z?usp=sharing)

In [1]:
%cd ..

/home/ax37/code/py/rudallel


In [2]:
# !conda install -c anaconda cudatoolkit=10.2

In [3]:
# !pip install torch==1.10.0

In [ ]:
!pip uninstall -y rudalle
!pip install -e .  # rudalle==0.0.1rc8 > /dev/null

Found existing installation: rudalle 1.1.0rc0
Uninstalling rudalle-1.1.0rc0:


In [ ]:
#@param {type: "float"}
ALLOWED_MEMORY = 6.5 # 3.5 # choose your GPU memory in GB  
if ALLOWED_MEMORY < 4.5:
    DALLE_BS = 1
elif ALLOWED_MEMORY < 5.5:
    DALLE_BS = 2
elif ALLOWED_MEMORY < 6.5:
    DALLE_BS = 3
elif ALLOWED_MEMORY < 7.5:
    DALLE_BS = 4
elif ALLOWED_MEMORY < 8.5:
    DALLE_BS = 5
elif ALLOWED_MEMORY < 9.5:
    DALLE_BS = 6
elif ALLOWED_MEMORY < 10.5:
    DALLE_BS = 7
else:
    DALLE_BS = 8
    
print('ruDALL-E batch size:', DALLE_BS)

In [ ]:
import multiprocessing
import torch
from psutil import virtual_memory


ram_gb = round(virtual_memory().total / 1024**3, 1)
print('CPU:', multiprocessing.cpu_count())
print('RAM GB:', ram_gb)
print("PyTorch version:", torch.__version__)
print("CUDA version:", torch.version.cuda)
print("cuDNN version:", torch.backends.cudnn.version())

!nvidia-smi

In [ ]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [ ]:
import transformers
import more_itertools
from tqdm.auto import tqdm

from rudalle.pipelines import generate_images, show  # , cherry_pick_by_clip
from rudalle import get_rudalle_model, get_tokenizer, get_vae  # , get_ruclip
from rudalle.utils import seed_everything, torch_tensors_to_pil_list

In [ ]:
# # prepare models:
device = "cuda"
dalle = get_rudalle_model("Malevich", pretrained=True, fp16=True, device=device)
tokenizer = get_tokenizer()
vae = get_vae(dwt=True)

# # prepare utils:
# ruclip, ruclip_processor = get_ruclip("ruclip-vit-base-patch32-v5")

In [ ]:
from rudalle.dalle.transformer import ELAttention

In [ ]:
import torch
import gc


gc.collect()
torch.cuda.empty_cache()
# torch.set_grad_enabled(False)
# att1 = dalle.module.transformer.layers[0].attention
# att1.custom_relax = False
# att1.cogview_pb_relax = False
# att1.eval()
# att1.attention_dropout = torch.nn.Identity()
# att2 = ELAttention(att1)
# torch.manual_seed(42)
# inputs = (torch.randn((2, 4, dalle.module.transformer.layers[0].attention.hidden_size)).cuda().half(),
#           torch.ones((1, 1, 4, 4)).cuda().half())
# x1, _ = att1(*inputs, use_cache=True)
# x2, _ = att2(*inputs, use_cache=True)
# diff = x1 - x2
# print((diff ** 2).sum())

# diff = diff.clamp(-4, 4)

# from matplotlib import pyplot as plt
# plt.imshow(diff.reshape(-1, diff.shape[-1] // 16).detach().float().cpu().numpy() ** 2)
# plt.colorbar()

In [ ]:
vae.cuda()
0

In [ ]:
with torch.cuda.amp.autocast():
    dalle.module.transformer.to_el()
0

In [ ]:
type(dalle.module.transformer.layers[0].attention)

In [ ]:
text = "эль-внимание"

seed_everything(14)
images = []
for top_k, top_p, images_num in [
    (2048, 0.995, 8),
#     (1536, 0.99, 8),
#     (1024, 0.99, 8),
#     (1024, 0.98, 8),
#     (512, 0.97, 8),
#     (384, 0.96, 8),
#     (256, 0.95, 8),
#     (128, 0.95, 8),
]:
    images += generate_images(text, tokenizer, dalle, vae,
                                 top_k=top_k, images_num=images_num,
                                 top_p=top_p, bs=DALLE_BS,
                                 use_cache=True)
show(images[0])

In [ ]:
show(images[0])

In [ ]:
pil_images = []
for _codebooks in tqdm(torch.cat(codebooks).cpu()):
    with torch.no_grad():
        images = vae.decode(_codebooks.unsqueeze(0))
        pil_images += torch_tensors_to_pil_list(images)